In [4]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [8]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [9]:
subject = 'Tocantins - Consumo de Cimento (t)'
split_index = 17 #Referente aos 18 anos de input  

In [10]:
data = pd.read_csv('2003_01_model_input_TO.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Ano,Tocantins - IDH,Tocantins - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Tocantins - Produção de Cimento (t),Tocantins - PIB - Estadual,Tocantins - PIB - Construção Civil,Tocantins - PIB - Per Capita,Tocantins - PIB - Preços de Mercado,Tocantins - Consumo de Cimento (t)
0,2003,0.700107,8.243830,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,0.895303,152.796000,1.593908e+07,1.457380e+06,10.276585,2.572692e+07,239.493000
1,2004,0.704664,8.173050,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,0.746570,152.796000,1.620310e+07,1.477171e+06,10.336645,2.614514e+07,397.426000
2,2005,0.708752,8.152621,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,0.589811,152.796000,1.645829e+07,1.491627e+06,10.399123,2.656860e+07,431.203000
3,2006,0.712378,8.067823,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,0.606044,152.796000,1.671172e+07,1.511199e+06,10.474837,2.700930e+07,312.507000
4,2007,0.715551,8.043255,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,0.633401,156.499264,1.694896e+07,1.533652e+06,10.545712,2.744722e+07,426.737000
5,2008,0.718281,8.013079,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,0.587619,199.994423,1.715433e+07,1.552702e+06,10.598411,2.786493e+07,450.745000
6,2009,0.720575,8.041614,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,0.566759,305.829017,1.731978e+07,1.569541e+06,10.617459,2.826546e+07,467.588000
7,2010,0.722443,8.006059,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,0.593537,421.327526,1.782790e+07,1.443676e+06,10.617472,2.605538e+07,487.208000
8,2011,0.723894,7.993875,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,0.600821,483.313605,1.829602e+07,1.330737e+06,10.632146,2.434482e+07,458.331000
9,2012,0.724531,9.385999,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,0.604731,510.546944,1.873679e+07,1.258451e+06,10.650202,2.323840e+07,508.683000


In [11]:
input_data = data.iloc[:split_index + 1, 1:-1]
input_data

,Tocantins - IDH,Tocantins - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Tocantins - Produção de Cimento (t),Tocantins - PIB - Estadual,Tocantins - PIB - Construção Civil,Tocantins - PIB - Per Capita,Tocantins - PIB - Preços de Mercado
0,0.700107,8.243830,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,0.895303,152.796000,1.593908e+07,1.457380e+06,10.276585,2.572692e+07
1,0.704664,8.173050,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,0.746570,152.796000,1.620310e+07,1.477171e+06,10.336645,2.614514e+07
2,0.708752,8.152621,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,0.589811,152.796000,1.645829e+07,1.491627e+06,10.399123,2.656860e+07
3,0.712378,8.067823,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,0.606044,152.796000,1.671172e+07,1.511199e+06,10.474837,2.700930e+07
4,0.715551,8.043255,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,0.633401,156.499264,1.694896e+07,1.533652e+06,10.545712,2.744722e+07
5,0.718281,8.013079,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,0.587619,199.994423,1.715433e+07,1.552702e+06,10.598411,2.786493e+07
6,0.720575,8.041614,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,0.566759,305.829017,1.731978e+07,1.569541e+06,10.617459,2.826546e+07
7,0.722443,8.006059,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,0.593537,421.327526,1.782790e+07,1.443676e+06,10.617472,2.605538e+07
8,0.723894,7.993875,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,0.600821,483.313605,1.829602e+07,1.330737e+06,10.632146,2.434482e+07
9,0.724531,9.385999,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,0.604731,510.546944,1.873679e+07,1.258451e+06,10.650202,2.323840e+07


In [12]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0     397.426000
1     431.203000
2     312.507000
3     426.737000
4     450.745000
5     467.588000
6     487.208000
7     458.331000
8     508.683000
9     525.234000
10    426.767286
11    454.929429
12    489.016286
13    500.270000
14    396.438000
15    402.932000
16    500.126000
17    542.030000
18    554.321000
19           NaN
Name: Tocantins - Consumo de Cimento (t), dtype: float64

In [13]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Tocantins - IDH,Tocantins - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Tocantins - Produção de Cimento (t),Tocantins - PIB - Estadual,Tocantins - PIB - Construção Civil,Tocantins - PIB - Per Capita,Tocantins - PIB - Preços de Mercado
0,-2.063352,-0.741308,2.057317,3.335319,2.503268,1.984588,-1.915550,-2.217697,2.670619,-1.292927,-1.586193,0.642178,-2.224300,0.336980
1,-1.629211,-0.814666,1.041262,1.436308,0.974434,0.428265,-1.603622,-1.796936,1.398420,-1.292927,-1.402007,0.798865,-1.818145,0.568862
2,-1.239842,-0.835840,-0.382310,0.608392,1.584406,-0.633957,-1.305339,-1.387550,0.057571,-1.292927,-1.223985,0.913318,-1.395643,0.803645
3,-0.894416,-0.923727,-0.655123,0.362019,0.700171,0.047872,-1.020736,-0.988421,0.196419,-1.292927,-1.047192,1.068275,-0.883632,1.047994
4,-0.592103,-0.949190,-0.562655,0.214378,-0.043890,-0.213672,-0.749851,-0.597180,0.430422,-1.272791,-0.881690,1.246041,-0.404347,1.290794
5,-0.332074,-0.980466,-0.668618,-0.171237,0.095846,-0.882129,-0.492719,-0.230290,0.038818,-1.036291,-0.738420,1.396863,-0.047977,1.522389
6,-0.113500,-0.950891,-0.307381,-0.482640,-0.506598,-0.409438,-0.249377,0.100246,-0.139605,-0.460827,-0.623003,1.530180,0.080835,1.744465
7,0.064448,-0.987743,-0.103220,-0.675189,-0.543480,-0.165137,0.025012,0.397049,0.089437,0.167184,-0.268534,0.533683,0.080926,0.519094
8,0.202599,-1.000370,-0.057995,-0.608258,-0.108258,-0.384906,0.201698,0.638927,0.151742,0.504226,0.058031,-0.360483,0.180154,-0.429315
9,0.263337,0.442479,0.191223,-0.550547,-0.852408,-0.436543,0.383423,0.817429,0.185184,0.652305,0.365519,-0.932794,0.302256,-1.042766


In [14]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0     397.426000
1     431.203000
2     312.507000
3     426.737000
4     450.745000
5     467.588000
6     487.208000
7     458.331000
8     508.683000
9     525.234000
10    426.767286
11    454.929429
12    489.016286
13    500.270000
14    396.438000
15    402.932000
16    500.126000
Name: Tocantins - Consumo de Cimento (t), dtype: float64

In [15]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Tocantins - IDH,Tocantins - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Tocantins - Produção de Cimento (t),Tocantins - PIB - Estadual,Tocantins - PIB - Construção Civil,Tocantins - PIB - Per Capita,Tocantins - PIB - Preços de Mercado
17,0.988301,0.41331,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,1.668012,0.922327,-0.585061,0.797291,-0.390079


In [16]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:split_index + 1]
test_target

17    542.03
Name: Tocantins - Consumo de Cimento (t), dtype: float64

In [17]:
train_input.iloc[-(len(train_input)//5):]

,Tocantins - IDH,Tocantins - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Tocantins - Produção de Cimento (t),Tocantins - PIB - Estadual,Tocantins - PIB - Construção Civil,Tocantins - PIB - Per Capita,Tocantins - PIB - Preços de Mercado
14,1.344748,0.969481,-1.195037,-0.553813,-0.504695,-0.765492,1.098789,0.71401,-1.705322,1.034608,1.234161,-0.902862,1.074777,-0.777508
15,1.309097,0.831381,-1.670834,-0.541784,-1.357653,0.164210,1.226007,0.54710,-1.702696,1.137673,1.225105,-0.810039,1.090228,-0.652844
16,1.192264,0.682584,-0.776357,-0.721152,-1.478508,3.015562,1.346999,0.33274,0.058929,1.436419,1.160169,-0.725116,1.061181,-0.519893


In [18]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    validation_size = len(train_input)//5
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=10000,
                      validation_data=(train_input.iloc[-validation_size:], 
                                       train_target.iloc[-validation_size:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [19]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [21]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[1559638101, 3315566925, 1687962309, 3158514496, 1786233301, 879987438, 2141847488, 1596980441, 2985146946, 4223559701, 3404489213, 2136955692, 2005191659, 2516473417, 4011053276, 1181040537, 3444632653, 2231866535, 1104273397, 1001977197, 3765505779, 1241694152, 2375568556, 2659886425, 354636043, 2862566056, 673047204, 3302495761, 1331800837, 1104602659, 3149792104, 3782721757, 583930616, 522344957, 3987981968, 1043144208, 734904230, 1482552428, 4165900569, 2865588327, 1002026358, 1179689965, 238096604, 1585196152, 1923526053, 2584560163, 4067125147, 3865594744, 2487481844, 1323102654, 3566188138, 4043298520, 155135316, 1389316082, 3127019531, 858304137, 1441373540, 2324233233, 3407312497, 1271571064, 1608318206, 3402372270, 2942097136, 623804719, 3727167558, 471056340, 3112680859, 1978789033, 3632031946, 2611855751, 3545813245, 3916454833, 93686029, 715803281, 1837857761, 4246614091, 2022837285, 554496722, 3841939772, 2482918053, 2628636298, 588728078, 788125407, 3480668620, 16391728

val_loss: 2301.48828125


Step: 85 ___________________________________________
val_loss: 3132.458251953125


Step: 86 ___________________________________________
val_loss: 2809.463623046875


Step: 87 ___________________________________________
val_loss: 2777.108154296875


Step: 88 ___________________________________________
val_loss: 2776.190673828125


Step: 89 ___________________________________________
val_loss: 2308.818115234375


Step: 90 ___________________________________________
val_loss: 2394.162353515625


Step: 91 ___________________________________________
val_loss: 2794.508544921875


Step: 92 ___________________________________________
val_loss: 3221.970458984375


Step: 93 ___________________________________________
val_loss: 2972.945068359375


Step: 94 ___________________________________________
val_loss: 2491.089111328125


Step: 95 ___________________________________________
val_loss: 2797.207763671875


Step: 96 ___________________________________________
val_loss

In [22]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 0s 447ms/step - loss: 204441.8438 - val_loss: 196558.8906
Epoch 2/10000
1/1 [==============================] - 0s 14ms/step - loss: 194673.5312 - val_loss: 57567.4883
Epoch 3/10000
1/1 [==============================] - 0s 14ms/step - loss: 197667.9688 - val_loss: 121413.5547
Epoch 4/10000
1/1 [==============================] - 0s 14ms/step - loss: 184024.5469 - val_loss: 99428.8672
Epoch 5/10000
1/1 [==============================] - 0s 14ms/step - loss: 180083.9844 - val_loss: 103190.0234
Epoch 6/10000
1/1 [==============================] - 0s 14ms/step - loss: 176607.5469 - val_loss: 99407.6328
Epoch 7/10000
1/1 [==============================] - 0s 13ms/step - loss: 173242.7031 - val_loss: 97788.6250
Epoch 8/10000
1/1 [==============================] - 0s 13ms/step - loss: 169946.2188 - val_loss: 95625.0938
Epoch 9/10000
1/1 [==============================] - 0s 14ms/step - loss: 166713.5938 - val_loss: 93665.7266
Epoch 10/10000


Epoch 76/10000
1/1 [==============================] - 0s 14ms/step - loss: 46892.5312 - val_loss: 22309.9375
Epoch 77/10000
1/1 [==============================] - 0s 14ms/step - loss: 46032.2148 - val_loss: 21833.3262
Epoch 78/10000
1/1 [==============================] - 0s 13ms/step - loss: 45188.4805 - val_loss: 21366.9844
Epoch 79/10000
1/1 [==============================] - 0s 14ms/step - loss: 44361.0195 - val_loss: 20910.6973
Epoch 80/10000
1/1 [==============================] - 0s 15ms/step - loss: 43549.5000 - val_loss: 20464.2598
Epoch 81/10000
1/1 [==============================] - 0s 13ms/step - loss: 42753.6289 - val_loss: 20027.4629
Epoch 82/10000
1/1 [==============================] - 0s 14ms/step - loss: 41973.0977 - val_loss: 19600.1367
Epoch 83/10000
1/1 [==============================] - 0s 13ms/step - loss: 41207.6172 - val_loss: 19182.0586
Epoch 84/10000
1/1 [==============================] - 0s 15ms/step - loss: 40456.8945 - val_loss: 18773.0664
Epoch 85/10000
1/1 

Epoch 151/10000
1/1 [==============================] - 0s 14ms/step - loss: 6221.6362 - val_loss: 2214.7605
Epoch 152/10000
1/1 [==============================] - 0s 14ms/step - loss: 6920.5454 - val_loss: 2471.0886
Epoch 153/10000
1/1 [==============================] - 0s 13ms/step - loss: 5136.0742 - val_loss: 3121.1677
Epoch 154/10000
1/1 [==============================] - 0s 13ms/step - loss: 8357.1836 - val_loss: 2411.8618
Epoch 155/10000
1/1 [==============================] - 0s 13ms/step - loss: 7399.7227 - val_loss: 5721.2129
Epoch 156/10000
1/1 [==============================] - 0s 13ms/step - loss: 7077.8579 - val_loss: 1882.1245
Epoch 157/10000
1/1 [==============================] - 0s 13ms/step - loss: 4944.4219 - val_loss: 2026.3887
Epoch 158/10000
1/1 [==============================] - 0s 14ms/step - loss: 4836.5386 - val_loss: 1997.3580
Epoch 159/10000
1/1 [==============================] - 0s 13ms/step - loss: 5755.4419 - val_loss: 2160.1677
Epoch 160/10000
1/1 [=======

Epoch 227/10000
1/1 [==============================] - 0s 15ms/step - loss: 1226.8503 - val_loss: 2758.0901
Epoch 228/10000
1/1 [==============================] - 0s 14ms/step - loss: 1634.9592 - val_loss: 2016.5580
Epoch 229/10000
1/1 [==============================] - 0s 14ms/step - loss: 1024.0062 - val_loss: 3167.5447
Epoch 230/10000
1/1 [==============================] - 0s 14ms/step - loss: 1548.3082 - val_loss: 1670.1571
Epoch 231/10000
1/1 [==============================] - 0s 14ms/step - loss: 1482.3525 - val_loss: 2686.8220
Epoch 232/10000
1/1 [==============================] - 0s 13ms/step - loss: 1917.2792 - val_loss: 1515.3783
Epoch 233/10000
1/1 [==============================] - 0s 14ms/step - loss: 2302.0925 - val_loss: 432.9775
Epoch 234/10000
1/1 [==============================] - 0s 14ms/step - loss: 1707.6938 - val_loss: 291.7536
Epoch 235/10000
1/1 [==============================] - 0s 14ms/step - loss: 1275.6888 - val_loss: 2936.1338
Epoch 236/10000
1/1 [=========

1/1 [==============================] - 0s 14ms/step - loss: 1242.5283 - val_loss: 2927.1711
Epoch 304/10000
1/1 [==============================] - 0s 14ms/step - loss: 3074.2729 - val_loss: 2387.4749
Epoch 305/10000
1/1 [==============================] - 0s 13ms/step - loss: 2090.7776 - val_loss: 2417.9060
Epoch 306/10000
1/1 [==============================] - 0s 13ms/step - loss: 1680.9446 - val_loss: 2255.1406
Epoch 307/10000
1/1 [==============================] - 0s 13ms/step - loss: 1783.9080 - val_loss: 2427.5208
Epoch 308/10000
1/1 [==============================] - 0s 13ms/step - loss: 1566.4395 - val_loss: 2096.5720
Epoch 309/10000
1/1 [==============================] - 0s 13ms/step - loss: 1416.5963 - val_loss: 2155.3931
Epoch 310/10000
1/1 [==============================] - 0s 13ms/step - loss: 1339.6833 - val_loss: 2105.7678
Epoch 311/10000
1/1 [==============================] - 0s 13ms/step - loss: 1289.9296 - val_loss: 2706.1301
Epoch 312/10000
1/1 [=======================

...

1/1 [==============================] - 0s 13ms/step - loss: 1308.7762 - val_loss: 227.8755
Epoch 837/10000
1/1 [==============================] - 0s 13ms/step - loss: 1866.7386 - val_loss: 291.2207
Epoch 838/10000
1/1 [==============================] - 0s 13ms/step - loss: 1407.8394 - val_loss: 367.7740
Epoch 839/10000
1/1 [==============================] - 0s 13ms/step - loss: 1312.4750 - val_loss: 364.9684
Epoch 840/10000
1/1 [==============================] - 0s 13ms/step - loss: 1342.2407 - val_loss: 239.6283
Epoch 841/10000
1/1 [==============================] - 0s 13ms/step - loss: 1364.8983 - val_loss: 333.9827
Epoch 842/10000
1/1 [==============================] - 0s 13ms/step - loss: 1428.3785 - val_loss: 192.8433
Epoch 843/10000
1/1 [==============================] - 0s 14ms/step - loss: 1297.8486 - val_loss: 213.3145
Epoch 844/10000
1/1 [==============================] - 0s 14ms/step - loss: 1282.9025 - val_loss: 157.8580
Epoch 845/10000
1/1 [==============================] 

Epoch 913/10000
1/1 [==============================] - 0s 13ms/step - loss: 984.4088 - val_loss: 139.2214
Epoch 914/10000
1/1 [==============================] - 0s 13ms/step - loss: 1114.6991 - val_loss: 130.2921
Epoch 915/10000
1/1 [==============================] - 0s 14ms/step - loss: 1229.1182 - val_loss: 229.7885
Epoch 916/10000
1/1 [==============================] - 0s 14ms/step - loss: 1121.8156 - val_loss: 133.0181
Epoch 917/10000
1/1 [==============================] - 0s 14ms/step - loss: 1128.2072 - val_loss: 132.8169
Epoch 918/10000
1/1 [==============================] - 0s 13ms/step - loss: 1227.6064 - val_loss: 178.2632
Epoch 919/10000
1/1 [==============================] - 0s 14ms/step - loss: 1125.5109 - val_loss: 133.0952
Epoch 920/10000
1/1 [==============================] - 0s 13ms/step - loss: 1218.5901 - val_loss: 192.3435
Epoch 921/10000
1/1 [==============================] - 0s 13ms/step - loss: 1208.5758 - val_loss: 154.8546
Epoch 922/10000
1/1 [=================

1/1 [==============================] - 0s 13ms/step - loss: 1184.3547 - val_loss: 475.7111
Epoch 990/10000
1/1 [==============================] - 0s 13ms/step - loss: 1355.4377 - val_loss: 542.9301
Epoch 991/10000
1/1 [==============================] - 0s 13ms/step - loss: 1107.4041 - val_loss: 506.2319
Epoch 992/10000
1/1 [==============================] - 0s 13ms/step - loss: 1347.8673 - val_loss: 543.7598
Epoch 993/10000
1/1 [==============================] - 0s 14ms/step - loss: 1115.4041 - val_loss: 383.1334
Epoch 994/10000
1/1 [==============================] - 0s 13ms/step - loss: 1353.4811 - val_loss: 455.3506
Epoch 995/10000
1/1 [==============================] - 0s 13ms/step - loss: 1124.4290 - val_loss: 375.1302
Epoch 996/10000
1/1 [==============================] - 0s 14ms/step - loss: 1076.4132 - val_loss: 377.0695
Epoch 997/10000
1/1 [==============================] - 0s 13ms/step - loss: 1088.0216 - val_loss: 365.7418
Epoch 998/10000
1/1 [==============================] 

1/1 [==============================] - 0s 13ms/step - loss: 1099.8198 - val_loss: 361.4319
Epoch 1066/10000
1/1 [==============================] - 0s 13ms/step - loss: 1101.2775 - val_loss: 361.3812
Epoch 1067/10000
1/1 [==============================] - 0s 14ms/step - loss: 988.8843 - val_loss: 348.4242
Epoch 1068/10000
1/1 [==============================] - 0s 13ms/step - loss: 1022.4775 - val_loss: 255.3637
Epoch 1069/10000
1/1 [==============================] - 0s 13ms/step - loss: 1036.5790 - val_loss: 259.7673
Epoch 1070/10000
1/1 [==============================] - 0s 14ms/step - loss: 1065.8582 - val_loss: 340.0689
Epoch 1071/10000
1/1 [==============================] - 0s 13ms/step - loss: 1040.8608 - val_loss: 326.9220
Epoch 1072/10000
1/1 [==============================] - 0s 13ms/step - loss: 1054.8439 - val_loss: 334.9898
Epoch 1073/10000
1/1 [==============================] - 0s 13ms/step - loss: 1044.0961 - val_loss: 269.2070
Epoch 1074/10000
1/1 [========================

1/1 [==============================] - 0s 13ms/step - loss: 987.5063 - val_loss: 132.3827
Epoch 1142/10000
1/1 [==============================] - 0s 14ms/step - loss: 838.2168 - val_loss: 132.6820
Epoch 1143/10000
1/1 [==============================] - 0s 13ms/step - loss: 994.2949 - val_loss: 130.7607
Epoch 1144/10000
1/1 [==============================] - 0s 13ms/step - loss: 984.4863 - val_loss: 128.2102
Epoch 1145/10000
1/1 [==============================] - 0s 13ms/step - loss: 975.8038 - val_loss: 119.8759
Epoch 1146/10000
1/1 [==============================] - 0s 13ms/step - loss: 989.0922 - val_loss: 125.5054
Epoch 1147/10000
1/1 [==============================] - 0s 13ms/step - loss: 978.5025 - val_loss: 116.6343
Epoch 1148/10000
1/1 [==============================] - 0s 13ms/step - loss: 975.3798 - val_loss: 116.8133
Epoch 1149/10000
1/1 [==============================] - 0s 13ms/step - loss: 995.0845 - val_loss: 111.7404
Epoch 1150/10000
1/1 [==============================] 

In [23]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 47ms/step
[542.03](test_target) - [[495.55444]](prediction) = 46.47555664062497


In [24]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [25]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 9, winner_seed)
predictions

train_input:


,Tocantins - IDH,Tocantins - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Tocantins - Produção de Cimento (t),Tocantins - PIB - Estadual,Tocantins - PIB - Construção Civil,Tocantins - PIB - Per Capita,Tocantins - PIB - Preços de Mercado
0,-1.727467,1.867665,2.184461,2.272299,1.914650,2.360960,-1.590751,-1.593283,2.299346,-0.631962,-1.518951,-1.118060,-1.656600,-1.351888
1,-1.109372,0.993164,1.077411,0.707092,0.380286,0.491065,-1.095308,-1.106645,0.891536,-0.631962,-1.063904,-0.649645,-1.175234,-0.863830
2,-0.555018,0.740755,-0.473651,0.024704,0.992464,-0.785177,-0.621537,-0.633164,-0.592240,-0.631962,-0.624086,-0.307492,-0.674494,-0.369666
3,-0.063227,-0.306945,-0.770896,-0.178361,0.105030,0.034030,-0.169496,-0.171546,-0.438592,-0.631962,-0.187304,0.155750,-0.067670,0.144631
4,0.367183,-0.610487,-0.670147,-0.300051,-0.641723,-0.280211,0.260758,0.280950,-0.179645,-0.592334,0.221582,0.687181,0.500368,0.655668
5,0.737392,-0.983325,-0.785600,-0.617883,-0.501481,-1.083350,0.669167,0.705281,-0.612992,-0.126908,0.575542,1.138060,0.922730,1.143124
6,1.048580,-0.630764,-0.392011,-0.874548,-1.106104,-0.515420,1.055674,1.087568,-0.810435,1.005591,0.860688,1.536610,1.075396,1.610541
7,1.301929,-1.070063,-0.169566,-1.033252,-1.143120,-0.221898,1.491492,1.430840,-0.556978,2.241499,1.736433,-1.442403,1.075503,-0.968580


train_target:


,Tocantins - Consumo de Cimento (t)
0,397.426
1,431.203
2,312.507
3,426.737
4,450.745
5,467.588
6,487.208
7,458.331


test_input:


...

,Tocantins - IDH,Tocantins - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Tocantins - Produção de Cimento (t),Tocantins - PIB - Estadual,Tocantins - PIB - Construção Civil,Tocantins - PIB - Per Capita,Tocantins - PIB - Preços de Mercado
17,0.988301,0.41331,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,1.668012,0.922327,-0.585061,0.797291,-0.390079


test_target:


,Tocantins - Consumo de Cimento (t)
17,542.03


1/1 [==============================] - 0s 35ms/step
Error: 46.47555664062497




[458.3504943847656,
 351.81634521484375,
 516.9715576171875,
 476.0061340332031,
 458.6713562011719,
 471.6076354980469,
 494.18359375,
 461.9137268066406,
 446.2225036621094,
 495.554443359375]

In [26]:
display(targets)
display(predictions)

[508.683,
 525.234,
 426.7672857142857,
 454.92942857142856,
 489.0162857142857,
 500.27,
 396.438,
 402.932,
 500.126,
 542.03]

[458.3504943847656,
 351.81634521484375,
 516.9715576171875,
 476.0061340332031,
 458.6713562011719,
 471.6076354980469,
 494.18359375,
 461.9137268066406,
 446.2225036621094,
 495.554443359375]

In [27]:
mae = mean_absolute_error(predictions, targets)
mae

65.11448053152901

In [28]:
porcentage = mae/np.mean(targets)
porcentage

0.13718633879792716